In [1]:
import xarray
import numpy as np
import pandas as pd

In [2]:
max_year = 5
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)

luh2_states_worldwide = luh2_states.isel(time=slice(1120, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1970-01-01", end="2022-01-01", freq='YS')
luh2_states_worldwide = luh2_states_worldwide.drop(['urban'])

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
transition_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/transitions.nc'
luh2_trans = xarray.open_dataset(transition_LUH2, engine="netcdf4", decode_times=False)

luh2_trans_worldwide = luh2_trans.isel(time=slice(1120, 1127))
luh2_trans_worldwide['time'] = pd.date_range(start='1970-01-01', end='1976-01-01', freq='YS')
luh2_trans_worldwide = luh2_trans_worldwide.drop(['urban_to_range', 'primf_to_urban', 'primn_to_urban', 'secdf_to_urban', 'secdn_to_urban', 'primf_bioh', 'primn_bioh','secmf_bioh','secyf_bioh','secnf_bioh', "urban_to_secdf", "urban_to_secdn","urban_to_c3ann","urban_to_c4ann", "urban_to_c3per",
    "urban_to_c4per", "urban_to_c3nfx", "urban_to_pastr", "c3ann_to_urban", "c4ann_to_urban", "c3per_to_urban", "c4per_to_urban", "c3nfx_to_urban", "pastr_to_urban", "range_to_urban"])

In [4]:
## add the projection

luh2_added_states_worldwide = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp119-2-1-f_gn_2015-2100.nc",\
                                        engine="netcdf4", decode_times=False)
luh2_added_trans = luh2_added_states_worldwide.drop_vars(['lat_bounds', 'lon_bounds', 'time_bnds'])
luh2_added_trans.coords['time'] = pd.date_range(start='2015-01-01', end='2100-01-01', freq='YS')

static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

staticcode = 'D:/kerja/asisten riset/code v/gpw_v4_national_identifier_grid_rev11_15_min_finall.nc'
luh2_static2 = xarray.open_dataset(staticcode, engine="netcdf4")

luh2_static = xarray.merge([luh2_static2, luh2_static], join='right', compat="override")

In [5]:
# ### Convert **ccode** to Country Name

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")

ccode_iso = list(country_code['country-code'])
csub_iso = list(country_code['sub-region'])
cregion_iso = list(country_code['region'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((720, 1440), dtype="<U64")
ccode_convert_cregion = np.zeros((720, 1440), dtype="<U64")
ccode_convert_sub = np.zeros((720, 1440), dtype="<U64")

for i in range(720):
    for j in range(1440):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
            ccode_convert_cregion[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
            ccode_convert_sub[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "Ocean"
            ccode_convert_cregion[i][j] = "Ocean"
            ccode_convert_sub[i][j] = "Ocean"

            
country = xarray.Dataset({"country": (["lat", "lon"], ccode_convert)},
                          coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})
subregion = xarray.Dataset({"sub-region": (["lat", "lon"], ccode_convert_sub)},
                          coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})
region = xarray.Dataset({"region": (["lat", "lon"], ccode_convert_cregion)},
                          coords={ "lon":luh2_states_worldwide.coords["lon"].to_numpy(), "lat":luh2_states_worldwide.coords["lat"].to_numpy()})

In [6]:
# # Equation Adaptation
forest_all_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4")
primf_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
secdf_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-SECDF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
primn_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMN-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
secdn_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-SECDN-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
c3ann_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
c4ann_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
c3per_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
c4per_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
c3nfx_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
range_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
pastr_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)


In [7]:
#General List
landuse_class_all = ['primf', 'primn', 'secdf', 'secdn', 'c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_prim_secd = ['primf', 'primn', 'secdf', 'secdn']
landuse_class_pastr_crop = ['c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_crops = ['c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx']
landuse_class_secd = ['secdf','secdn']
carstock_biomass = ['branches', 'leaves', 'stems', 'roots']
carstock_soil = ['litter', 'humus', 'charcoal']

land_clearing_to_crop = ['primf_to_c3ann','primf_to_c4ann','primf_to_c3per','primf_to_c4per','primf_to_c3nfx',
                        'primn_to_c3ann','primn_to_c4ann','primn_to_c3per','primn_to_c4per','primn_to_c3nfx',
                        'secdf_to_c3ann','secdf_to_c4ann','secdf_to_c3per','secdf_to_c4per','secdf_to_c3nfx',
                        'secdn_to_c3ann','secdn_to_c4ann','secdn_to_c3per','secdn_to_c4per','secdn_to_c3nfx']

land_clearing_to_pasture = ['primf_to_pastr', 'primf_to_range', 
                            'primn_to_pastr', 'primn_to_range',
                            'secdf_to_pastr', 'secdf_to_range',
                            'secdn_to_pastr', 'secdn_to_range']

ajl_clearing = ['primf', 'primn', 'secdf', 'secdn']

#List for FROM analysis
#Harvest
primf_harvest = ["primf_harv"]
primn_harvest = ["primn_harv"]
secmf_harvest = ["secmf_harv"]
secyf_harvest = ["secyf_harv"]
secdn_harvest = ["secnf_harv"]

# Other Transitions
# Pasture to Crop
pastr_to_crop = ['pastr_to_c3ann','pastr_to_c4ann','pastr_to_c3per','pastr_to_c4ann','pastr_to_c3nfx']
range_to_crop = ['range_to_c3ann','range_to_c4ann','range_to_c3per','range_to_c4ann','range_to_c3nfx']
# Crop to Crop
c3ann_to_crop = ['c3ann_to_c4ann','c3ann_to_c3per','c3ann_to_c4per','c3ann_to_c3nfx']
c4ann_to_crop = ['c4ann_to_c3ann','c4ann_to_c3per','c4ann_to_c4per','c4ann_to_c3nfx']
c3per_to_crop = ['c3per_to_c3ann','c3per_to_c4ann','c3per_to_c4per','c3per_to_c3nfx']
c4per_to_crop = ['c4per_to_c3ann','c4per_to_c4ann','c4per_to_c3per','c4per_to_c3nfx']
c3nfx_to_crop = ['c3nfx_to_c3ann','c3nfx_to_c4ann','c3nfx_to_c3per','c3nfx_to_c4per']

# Crop to Pasture
c3ann_to_pastr = ['c3ann_to_pastr','c3ann_to_range']
c4ann_to_pastr = ['c4ann_to_pastr','c4ann_to_range']
c3per_to_pastr = ['c3per_to_pastr','c3per_to_range']
c4per_to_pastr = ['c4per_to_pastr','c4per_to_range']
c3nfx_to_pastr = ['c3nfx_to_pastr','c3nfx_to_range']
# Pasture to Pasture
pastr_to_pasture = ['pastr_to_range']
range_to_pasture = ['range_to_pastr']
# Land Abandonment
c3ann_to_abandon = ['c3ann_to_secdf']
c4ann_to_abandon = ['c4ann_to_secdf']
c3per_to_abandon = ['c3per_to_secdf']
c4per_to_abandon = ['c4per_to_secdf']
c3nfx_to_abandon = ['c3nfx_to_secdf']
pastr_to_abandon = ['pastr_to_secdf']
range_to_abandon = ['range_to_secdf']


In [8]:
# Carbon Density
#Biomass
carbon_density_forest_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_primf_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_secdf_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_primn_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_secdn_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3nfx_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_range_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_pastr_bio = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_all_bio = np.zeros((max_year, 720, 1440), dtype="float32")

#Soil
carbon_density_forest_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_primf_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_secdf_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_primn_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_secdn_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c4per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_crop_c3nfx_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_range_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_pastr_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_all_soil = np.zeros((max_year, 720, 1440), dtype="float32")

# Carbon emission in every years
area_tf_harvest_clearing = np.zeros((max_year, 11, 720, 1440), dtype="float32")
carbon_emit_harv_biomass = np.zeros((max_year, 5, 720, 1440), dtype="float32")
carbon_emit_harv_soil = np.zeros((max_year, 5, 720, 1440), dtype="float32")

#master variable carbon emission
# temp_carbon_emit_trans_clear_bio = np.zeros((max_year, 56, 720, 1440), dtype="float32")
# temp_carbon_emit_trans_clear_soil = np.zeros((max_year, 56, 720, 1440), dtype="float32")

#1 carbon emission LUC to crops
#1a carbon emission forest to crops
#notes: we divide land clearing to 2 categories (crops and pastures); so maybe it is not need to have 56 dimension anymore, see initialization of LUC crops and LUC pasture list
temp_carbon_emit_trans_LUC_crops_bio = np.zeros((max_year, 20, 720, 1440), dtype="float32")
temp_carbon_emit_trans_LUC_crops_soil = np.zeros((max_year, 20, 720, 1440), dtype="float32")
#1b carbon emission pasture to crops
temp_carbon_emit_trans_pasturetocrop_bio = np.zeros((max_year, 20, 720, 1440), dtype="float32")
temp_carbon_emit_trans_pasturetocrop_soil = np.zeros((max_year, 20, 720, 1440), dtype="float32")
#1c carbon emission crops to crops
temp_carbon_emit_trans_croptocrop_bio = np.zeros((max_year, 40, 720, 1440), dtype="float32")
temp_carbon_emit_trans_croptocrop_soil = np.zeros((max_year, 40, 720, 1440), dtype="float32")

#2 carbon emission LUC to pasture
#2a carbon emission forest to pasture
#notes: we divide land clearing to 2 categories (crops and pastures); so maybe it is not need to have 56 dimension anymore, see initialization of LUC crops and LUC pasture list
temp_carbon_emit_trans_LUC_pasture_bio = np.zeros((max_year, 16, 720, 1440), dtype="float32")
temp_carbon_emit_trans_LUC_pasture_soil = np.zeros((max_year, 16, 720, 1440), dtype="float32")
#2b carbon emission crop to pasture
temp_carbon_emit_trans_croptopasture_bio = np.zeros((max_year, 20, 720, 1440), dtype="float32")
temp_carbon_emit_trans_croptopasture_soil = np.zeros((max_year, 20, 720, 1440), dtype="float32")
#2c carbon emission pasture to pasture
temp_carbon_emit_trans_pasturetopasture_bio = np.zeros((max_year, 4, 720, 1440), dtype="float32")
temp_carbon_emit_trans_pasturetopasture_soil = np.zeros((max_year, 4, 720, 1440), dtype="float32")

#3 carbon emission land abandonment
selisih_abandon = np.zeros((max_year, 720, 1440), dtype="float32")
# temp_carbon_emit_trans_abandon_bio = np.zeros((max_year, 14, 720, 1440), dtype="float32")
temp_carbon_emit_trans_abandon_bio = np.zeros((max_year, 10, 720, 1440), dtype="float32")
# temp_carbon_emit_trans_abandon_soil = np.zeros((max_year, 14, 720, 1440), dtype="float32")
temp_carbon_emit_trans_abandon_soil = np.zeros((max_year, 10, 720, 1440), dtype="float32")
selisih = np.zeros((max_year, 720, 1440), dtype="float32")

# Akumulatif carbon emission
#BIOMASS
temp_harvest_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
#temp_clear_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_crops_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_pasture_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_abandon_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptocrop_from_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptocrop_to_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptopasture_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetopasture_from_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetopasture_to_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetocrop_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")

#SOIL
temp_harvest_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
#temp_clear_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_crops_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_LUC_pasture_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_abandon_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptocrop_from_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptocrop_to_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_croptopasture_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetopasture_from_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetopasture_to_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_pasturetocrop_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")

#Agregasi untuk excel
luc_crops = np.zeros((max_year, 5, 720, 1440), dtype="float32")
luc_pasteur = np.zeros((max_year, 11, 720, 1440), dtype="float32")
wood_harvest = np.zeros((max_year, 11, 720, 1440), dtype="float32")
ag_abandon = np.zeros((max_year, 11, 720, 1440), dtype="float32")

area_updated_land_abandonment_bio = np.zeros((max_year, 720, 1440), dtype="float32")
area_updated_land_abandonment_soil = np.zeros((max_year, 720, 1440), dtype="float32")

all_to_c3ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_pastr_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_range_bio = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops_bio = np.zeros((max_year, 720, 1440), dtype="float32")

all_to_c3ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_pastr_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_range_soil = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops_soil = np.zeros((max_year, 720, 1440), dtype="float32")

all_to_c3ann = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4ann = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3per = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c4per = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_c3nfx = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_pastr = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_range = np.zeros((max_year, 720, 1440), dtype="float32")
all_to_crops = np.zeros((max_year, 720, 1440), dtype="float32")


In [9]:

#Assign the carbon density of biomass value to each year
for time_idx in range(max_year):
    for biomass_class in carstock_biomass:
        carbon_density_forest_bio[time_idx] += forest_all_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_primf_bio[time_idx] += primf_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_secdf_bio[time_idx] += secdf_carbon_density['secdf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_primn_bio[time_idx] += primn_carbon_density['primn_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_secdn_bio[time_idx] += secdn_carbon_density['secdn_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3ann_bio[time_idx] += c3ann_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c4ann_bio[time_idx] += c4ann_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3per_bio[time_idx] += c3per_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c4per_bio[time_idx] += c4per_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_crop_c3nfx_bio[time_idx] += c3nfx_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_range_bio[time_idx] += range_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_pastr_bio[time_idx] += pastr_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()

#I skipped the carbon density all since we want to focus on emission not the carbon equ before transition and after transition
#CARBON EMISSION CALCULATION

#HARVEST (hanya biomass saja, tidak soil karena keadaan soil tidak terganggu       
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for i, col_a in enumerate(landuse_class_prim_secd):
            if (i==0):
                for primf_harv_trans in primf_harvest:
                    carbon_emit_harv_biomass[time_idx][i] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                np.nan_to_num(luh2_trans_worldwide[primf_harv_trans].isel(time=time_idx).to_numpy()),
                                                                                carbon_density_primf_bio[time_idx]))
                    temp_harvest_increment_biomass[time_idx][i] += carbon_emit_harv_biomass[time_idx][i]
                
            if (i==1):
                for primn_harv_trans in primn_harvest:
                    carbon_emit_harv_biomass[time_idx][i] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                np.nan_to_num(luh2_trans_worldwide[primn_harv_trans].isel(time=time_idx).to_numpy()),
                                                                                carbon_density_primn_bio[time_idx]))
                    temp_harvest_increment_biomass[time_idx][i] += carbon_emit_harv_biomass[time_idx][i]
                
            if (i==2):
                for secmf_harv_trans in secmf_harvest:
                    carbon_emit_harv_biomass[time_idx][i] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                np.nan_to_num(luh2_trans_worldwide[secmf_harv_trans].isel(time=time_idx).to_numpy()),
                                                                                carbon_density_secdf_bio[time_idx]))
                    temp_harvest_increment_biomass[time_idx][i] += carbon_emit_harv_biomass[time_idx][i]
                for secyf_harv_trans in secyf_harvest:
                    carbon_emit_harv_biomass[time_idx][i+1] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                  np.nan_to_num(luh2_trans_worldwide[secyf_harv_trans].isel(time=time_idx).to_numpy()),
                                                                                  carbon_density_secdf_bio[time_idx]))
                    temp_harvest_increment_biomass[time_idx][i] += carbon_emit_harv_biomass[time_idx][i+1]
                
            if (i==3):
                for secnf_harv_trans in secdn_harvest:
                    carbon_emit_harv_biomass[time_idx][i+1] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                  np.nan_to_num(luh2_trans_worldwide[secnf_harv_trans].isel(time=time_idx).to_numpy()),
                                                                                  carbon_density_secdn_bio[time_idx]))
                    temp_harvest_increment_biomass[time_idx][i] += carbon_emit_harv_biomass[time_idx][i+1]


In [10]:

#FOREST TO CROP BIOMASS
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_to_crop):
                if (j == 0):
                    if (k <= 4):
                        temp_carbon_emit_trans_LUC_crops_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_bio[time_idx]))
                        if k == 0:
                            all_to_c3ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3ann_bio[time_idx]
                        elif k == 1:
                            all_to_c4ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4ann_bio[time_idx]
                        elif k == 2:
                            all_to_c3per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3per_bio[time_idx]
                        elif k == 3:
                            all_to_c4per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4per_bio[time_idx]
                        elif k == 4:
                            all_to_c3nfx_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3nfx_bio[time_idx]

                        temp_LUC_crops_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]

                elif (j == 1):
                    if (4 < k <= 9):
                        temp_carbon_emit_trans_LUC_crops_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_bio[time_idx]))
                        if k == 5:
                            all_to_c3ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3ann_bio[time_idx]
                        elif k == 6:
                            all_to_c4ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4ann_bio[time_idx]
                        elif k == 7:
                            all_to_c3per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3per_bio[time_idx]
                        elif k == 8:
                            all_to_c4per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4per_bio[time_idx]
                        elif k == 9:
                            all_to_c3nfx_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3nfx_bio[time_idx]

                        temp_LUC_crops_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]

                elif (j == 2):
                    if (9 < k <= 14):
                        temp_carbon_emit_trans_LUC_crops_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_bio[time_idx]))
                        if k == 10:
                            all_to_c3ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3ann_bio[time_idx]
                        elif k == 11:
                            all_to_c4ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4ann_bio[time_idx]
                        elif k == 12:
                            all_to_c3per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3per_bio[time_idx]
                        elif k == 13:
                            all_to_c4per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4per_bio[time_idx]
                        elif k == 14:
                            all_to_c3nfx_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3nfx_bio[time_idx]

                        temp_LUC_crops_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]

                elif (j == 3):
                    if (14 < k <= 19):
                        temp_carbon_emit_trans_LUC_crops_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_bio[time_idx]))
                        if k == 15:
                            all_to_c3ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3ann_bio[time_idx]
                        elif k == 16:
                            all_to_c4ann_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4ann_bio[time_idx]
                        elif k == 17:
                            all_to_c3per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3per_bio[time_idx]
                        elif k == 18:
                            all_to_c4per_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c4per_bio[time_idx]
                        elif k == 19:
                            all_to_c3nfx_bio[time_idx] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                            all_to_crops_bio[time_idx] += all_to_c3nfx_bio[time_idx]

                        temp_LUC_crops_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_crops_bio[time_idx][k]
                        
#FOREST TO PASTURE BIOMASS
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_to_pasture):
                if (j == 0):
                    if (k <= 1):
                        temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_bio[time_idx]))

                        temp_LUC_pasture_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k]

                elif (j == 1):
                    if (1 < k <= 3):
                        temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_bio[time_idx]))

                        temp_LUC_pasture_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k]

                elif (j == 2):
                    if (3 < k <= 5):
                        temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_bio[time_idx]))

                        temp_LUC_pasture_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k]

                elif (j == 3):
                    if (5 < k <= 7):
                        temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_bio[time_idx]))
                            
                        temp_LUC_pasture_increment_biomass[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_bio[time_idx][k]

# Land Abandonment to SECDF 
# BIOMASS

for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_pastr_crop):
            if (j == 0):
                for i,c3ann_trans_col in enumerate(c3ann_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3ann_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3ann_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]

            elif (j == 1):
                for i,c4ann_trans_col in enumerate(c4ann_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c4ann_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c4ann_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]

            elif (j == 2):
                for i,c3per_trans_col in enumerate(c3per_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3per_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3per_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]
                
            elif (j == 3):
                for i,c4per_trans_col in enumerate(c4per_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c4per_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c4per_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]
                
            elif (j == 4):
                for i,c3nfx_trans_col in enumerate(c3nfx_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3nfx_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3nfx_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]
                
            elif (j == 5):
                for i,pastr_trans_col in enumerate(pastr_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[pastr_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = (np.nan_to_num(carbon_density_pastr_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]
                
            elif (j == 6):
                for i,range_trans_col in enumerate(range_to_abandon):
                    area_updated_land_abandonment_bio[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[range_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = (np.nan_to_num(carbon_density_range_bio[time_idx])-(carbon_density_secdf_bio[time_idx+1]))
                    temp_carbon_emit_trans_abandon_bio[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_bio[time_idx]
                    temp_abandon_increment_biomass[time_idx][j] += temp_carbon_emit_trans_abandon_bio[time_idx][j]                       


In [11]:
#Assign the carbon density of soil value to each year
for time_idx in range(max_year):
    for soil_class in carstock_soil:
        carbon_density_forest_soil[time_idx] += forest_all_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_primf_soil[time_idx] += primf_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_secdf_soil[time_idx] += secdf_carbon_density['secdf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_primn_soil[time_idx] += primn_carbon_density['primn_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_secdn_soil[time_idx] += secdn_carbon_density['secdn_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3ann_soil[time_idx] += c3ann_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c4ann_soil[time_idx] += c4ann_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3per_soil[time_idx] += c3per_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c4per_soil[time_idx] += c4per_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_crop_c3nfx_soil[time_idx] += c3nfx_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_range_soil[time_idx] += range_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_pastr_soil[time_idx] += pastr_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()

#FOREST TO CROP SOIL
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_to_crop):
                if (j == 0):
                    if (k <= 4):
                        temp_carbon_emit_trans_LUC_crops_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_soil[time_idx]))
                        if k == 0:
                            all_to_c3ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3ann_soil[time_idx]
                        elif k == 1:
                            all_to_c4ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4ann_soil[time_idx]
                        elif k == 2:
                            all_to_c3per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3per_soil[time_idx]
                        elif k == 3:
                            all_to_c4per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4per_soil[time_idx]
                        elif k == 4:
                            all_to_c3nfx_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3nfx_soil[time_idx]

                        temp_LUC_crops_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]

                elif (j == 1):
                    if (4 < k <= 9):
                        temp_carbon_emit_trans_LUC_crops_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_soil[time_idx]))
                        if k == 5:
                            all_to_c3ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3ann_soil[time_idx]
                        elif k == 6:
                            all_to_c4ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4ann_soil[time_idx]
                        elif k == 7:
                            all_to_c3per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3per_soil[time_idx]
                        elif k == 8:
                            all_to_c4per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4per_soil[time_idx]
                        elif k == 9:
                            all_to_c3nfx_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3nfx_soil[time_idx]

                        temp_LUC_crops_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]

                elif (j == 2):
                    if (9 < k <= 14):
                        temp_carbon_emit_trans_LUC_crops_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_soil[time_idx]))
                        if k == 10:
                            all_to_c3ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3ann_soil[time_idx]
                        elif k == 11:
                            all_to_c4ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4ann_soil[time_idx]
                        elif k == 12:
                            all_to_c3per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3per_soil[time_idx]
                        elif k == 13:
                            all_to_c4per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4per_soil[time_idx]
                        elif k == 14:
                            all_to_c3nfx_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3nfx_soil[time_idx]

                        temp_LUC_crops_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]

                elif (j == 3):
                    if (14 < k <= 19):
                        temp_carbon_emit_trans_LUC_crops_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_soil[time_idx]))
                        if k == 15:
                            all_to_c3ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3ann_soil[time_idx]
                        elif k == 16:
                            all_to_c4ann_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4ann_soil[time_idx]
                        elif k == 17:
                            all_to_c3per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3per_soil[time_idx]
                        elif k == 18:
                            all_to_c4per_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c4per_soil[time_idx]
                        elif k == 19:
                            all_to_c3nfx_soil[time_idx] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                            all_to_crops_soil[time_idx] += all_to_c3nfx_soil[time_idx]

                        temp_LUC_crops_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_crops_soil[time_idx][k]
                        
#FOREST TO PASTURE SOIL
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_to_pasture):
                if (j == 0):
                    if (k <= 1):
                        temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_soil[time_idx]))
                        temp_LUC_pasture_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k]

                elif (j == 1):
                    if (1 < k <= 3):
                        temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_soil[time_idx]))
                        temp_LUC_pasture_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k]

                elif (j == 2):
                    if (3 < k <= 5):
                        temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_forest_soil[time_idx]))
                        temp_LUC_pasture_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k]

                elif (j == 3):
                    if (5 < k <= 7):
                        temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_trans_worldwide[colk].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_soil[time_idx]))
                        temp_LUC_pasture_increment_soil[time_idx][j] += temp_carbon_emit_trans_LUC_pasture_soil[time_idx][k]

##LAND ABANDONMENT TO SECDF
#SOIL
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_pastr_crop):
            if (j == 0):
                for i,c3ann_trans_col in enumerate(c3ann_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3ann_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3ann_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]

            elif (j == 1):
                for i,c4ann_trans_col in enumerate(c4ann_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c4ann_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c4ann_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]

            elif (j == 2):
                for i,c3per_trans_col in enumerate(c3per_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3per_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3per_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]
                
            elif (j == 3):
                for i,c4per_trans_col in enumerate(c4per_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c4per_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c4per_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]
                
            elif (j == 4):
                for i,c3nfx_trans_col in enumerate(c3nfx_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[c3nfx_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = ((carbon_density_crop_c3nfx_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]
                
            elif (j == 5):
                for i,pastr_trans_col in enumerate(pastr_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[pastr_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = (np.nan_to_num(carbon_density_pastr_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]
                
            elif (j == 6):
                for i,range_trans_col in enumerate(range_to_abandon):
                    area_updated_land_abandonment_soil[time_idx] = np.multiply(luh2_static['carea'].to_numpy(),np.nan_to_num(luh2_trans_worldwide[range_trans_col].isel(time=time_idx).to_numpy()))
                    selisih [time_idx] = (np.nan_to_num(carbon_density_range_soil[time_idx])-(carbon_density_secdf_soil[time_idx+1]))
                    temp_carbon_emit_trans_abandon_soil[time_idx][j] = selisih[time_idx] * area_updated_land_abandonment_soil[time_idx]
                    temp_abandon_increment_soil[time_idx][j] += temp_carbon_emit_trans_abandon_soil[time_idx][j]


In [12]:
for time_idx in range(max_year):
    all_to_c3ann[time_idx] += all_to_c3ann_bio[time_idx]+all_to_c3ann_soil[time_idx]
    all_to_c4ann[time_idx] += all_to_c4ann_bio[time_idx]+all_to_c4ann_soil[time_idx]
    all_to_c3per[time_idx] += all_to_c3per_bio[time_idx]+all_to_c3per_soil[time_idx]
    all_to_c4per[time_idx] += all_to_c4per_bio[time_idx]+all_to_c4per_soil[time_idx]
    all_to_c3nfx[time_idx] += all_to_c3nfx_bio[time_idx]+all_to_c3nfx_soil[time_idx]

In [13]:
for time_idx in range(max_year):
    #satuan dalam million tonne CO2
    all_to_c3ann[time_idx] = all_to_c3ann[time_idx]/1E9
    all_to_c4ann[time_idx] = all_to_c4ann[time_idx]/1E9
    all_to_c3per[time_idx] = all_to_c3per[time_idx]/1E9
    all_to_c4per[time_idx] = all_to_c4per[time_idx]/1E9
    all_to_c3nfx[time_idx] = all_to_c3nfx[time_idx]/1E9

In [17]:
new_c3ann = np.expand_dims(all_to_c3ann, axis=-1)
new_c4ann = np.expand_dims(all_to_c4ann, axis=-1)
new_c3per = np.expand_dims(all_to_c3per, axis=-1)
new_c4per = np.expand_dims(all_to_c4per, axis=-1)
new_c3nfx = np.expand_dims(all_to_c3nfx, axis=-1)

luc_crops = np.concatenate([new_c3ann, new_c4ann, new_c3per, new_c4per, new_c3nfx], axis=-1)

In [21]:
carbon_emission_luccrops = xarray.Dataset({
    "luc_crops": (["time", "lat", "lon", "cat"], luc_crops)
},
    coords={
        "time": pd.date_range(start="1970", end="1974", freq='ys'),
        "lat": luh2_states_worldwide.coords["lat"].to_numpy(),
        "lon": luh2_states_worldwide.coords["lon"].to_numpy(), 
        "cat": landuse_class_crops
    }
)
# carbon_emission_luccrops
carbon_emission_luccrops.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn_new.nc", mode='w', format="NETCDF4")

In [ ]:
carbon_emission_total = xarray.Dataset(
    coords={
        "time": pd.date_range(start="1/1/1970", end="1/1/1974", freq='ys'),
        "lat": luh2_states_worldwide.coords["lat"].to_numpy(),
        "lon": luh2_states_worldwide.coords["lon"].to_numpy(), 
        "country": country['country'],
        # "subregion": subregion['sub-region'],
        # "region": region['region']
    }
)
data_vars = {
    "all_to_c3ann_var": (["time", "lat", "lon"], all_to_c3ann),
    "all_to_c4ann_var": (["time", "lat", "lon"], all_to_c4ann),
    "all_to_c3per_var": (["time", "lat", "lon"], all_to_c3per),
    "all_to_c4per_var": (["time", "lat", "lon"], all_to_c4per),
    "all_to_c3nfx_var": (["time", "lat", "lon"], all_to_c3nfx)
}
carbon_emission_ntcdf = carbon_emission_total.assign(data_vars)
carbon_emission_ntcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn.nc", mode='w', format="NETCDF4")

In [2]:
import xarray
ds = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn_new.nc", engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:    (time: 5, lat: 720, lon: 1440, cat: 5)
Coordinates:
  * time       (time) datetime64[ns] 1970-01-01 1971-01-01 ... 1974-01-01
  * lat        (lat) float64 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon        (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * cat        (cat) <U5 'c3ann' 'c4ann' 'c3per' 'c4per' 'c3nfx'
Data variables:
    luc_crops  (time, lat, lon, cat) float32 ...

In [ ]:
# all_to_c3ann.sum() + all_to_c4ann.sum() + all_to_c3per.sum() + all_to_c4per.sum() + all_to_c3nfx.sum()

In [ ]:
# df = carbon_emission_ntcdf.to_dataframe()

In [ ]:
# df['all_to_c3ann_var'].sum() + df['all_to_c4ann_var'].sum() + df['all_to_c3per_var'].sum() + df['all_to_c4per_var'].sum() + df['all_to_c3nfx_var'].sum()

In [ ]:
# df_copy = df.copy()
# df_copy['all_to_c3ann_var'].sum() + df_copy['all_to_c4ann_var'].sum() + df_copy['all_to_c3nfx_var'].sum() + df_copy['all_to_c3per_var'].sum() + df_copy['all_to_c4per_var'].sum()

In [ ]:
# table = pd.pivot_table(df_copy, values=['all_to_c3ann_var', 'all_to_c4ann_var', 'all_to_c3per_var', 'all_to_c4per_var', 'all_to_c3nfx_var'],
#                        index=['country', 'subregion', 'region'], columns=['time'], aggfunc="sum", fill_value=0)

In [ ]:
# df_index = table.stack(level=0)
# data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
# df_index.columns = data.year
# df_index.index.set_names(['country', 'subregion', 'region', 'type'], inplace=True)
# df_index = df_index.reset_index()
# df_index

In [ ]:
# df_new_2 = df_copy.pivot_table(index=['all_to_c3ann_var', 'all_to_c4ann_var', 'all_to_c3per_var', 'all_to_c4per_var', 'all_to_c3nfx_var'], columns=['time'])
# df_new_2

In [ ]:
# df_index = df_new_2.stack(level=0)
# df_index

In [ ]:
# # df_index = df_new_2.stack(level=0)
# # data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
# # df_index.columns = data.year
# # df_index.index.set_names(['country', 'subregion', 'region', 'type'], inplace=True)
# df_index = df_index.reset_index()

In [ ]:
# df_index.to_excel('D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn_run12.xlsx', index=False)

In [ ]:
#Agregasi untuk excel
luc_crops = np.zeros((max_year, 11, 720, 1440), dtype="float32")
luc_pasteur = np.zeros((max_year, 11, 720, 1440), dtype="float32")
wood_harvest = np.zeros((max_year, 11, 720, 1440), dtype="float32")
ag_abandon = np.zeros((max_year, 11, 720, 1440), dtype="float32")

data_emission_crops = np.zeros((max_year, 720, 1440), dtype="float32")
data_emission_pasture = np.zeros((max_year, 720, 1440), dtype="float32")
data_emission_harvest = np.zeros((max_year, 720, 1440), dtype="float32")
data_emission_abandon = np.zeros((max_year, 720, 1440), dtype="float32")

In [ ]:
## AGREGASI DAN EXPORT EXCEL
for time_idx in range(max_year):
    luc_crops[time_idx] += temp_LUC_crops_increment_biomass[time_idx]+temp_LUC_crops_increment_soil[time_idx]
    luc_pasteur[time_idx] += temp_LUC_pasture_increment_biomass[time_idx]+temp_LUC_pasture_increment_soil[time_idx]
    ag_abandon[time_idx] += temp_abandon_increment_biomass[time_idx]+temp_abandon_increment_soil[time_idx]
    wood_harvest[time_idx] += temp_harvest_increment_biomass[time_idx]

In [ ]:
## AGREGASI DAN EXPORT EXCEL
for time_idx in range(max_year):
    luc_crops[time_idx] = luc_crops[time_idx] * 3.667 / 1E9
    luc_pasteur[time_idx] = luc_pasteur[time_idx]* 3.667 /1E9
    ag_abandon[time_idx] = ag_abandon[time_idx]* 3.667 /1E9
    wood_harvest[time_idx] = wood_harvest[time_idx]* 3.667 /1E9 

In [ ]:
# for time_idx in range(max_year):
#     for j in range(5):
#         data_emission_crops[time_idx] += luc_crops[time_idx][j]

# for time_idx in range(max_year):
#     for j in range(11):
#         data_emission_pasture[time_idx] += luc_pasteur[time_idx][j]
#         data_emission_abandon[time_idx] += ag_abandon[time_idx][j]
#         data_emission_harvest[time_idx] += wood_harvest[time_idx][j]


In [ ]:
carbon_emission_total = xarray.Dataset({
    "luc_crops": (["time", "cat", "lat", "lon"], luc_crops)
},
    coords={
        "time": pd.date_range(start="1970", end="1974", freq='ys'),
        "lat": luh2_states_worldwide.coords["lat"].to_numpy(),
        "lon": luh2_states_worldwide.coords["lon"].to_numpy(), 
        "cat": landuse_class_crops
    }
)
carbon_emission_ntcdf
# carbon_emission_ntcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_5CAT_5thn.nc", mode='w', format="NETCDF4")

In [ ]:
# carbon_emission_crops = xarray.Dataset(
#     coords={
#         "time": pd.date_range("1970-01-01", "1974-01-01", freq='YS'),
#         "lat": luh2_states_worldwide.coords["lat"].to_numpy(),
#         "lon": luh2_states_worldwide.coords["lon"].to_numpy(),
#         "country": country['country']
#     }
# )
# data_vars = {
#     "luc_crops": (["time", "lat", "lon"], data_emission_crops),
#     "luc_pasture": (["time", "lat", "lon"], data_emission_pasture),
#     "ag_abandon": (["time", "lat", "lon"], data_emission_abandon),
#     "wood_harvest": (["time", "lat", "lon"], data_emission_harvest)
# }
# carbon_emission_crops = carbon_emission_crops.assign(data_vars)
# carbon_emission_crops

In [ ]:
# df_2 = carbon_emission_crops.to_dataframe()

In [ ]:
# df_new_3 = df_2.copy()
# df_new_3 = df_new_3[(df_new_3["country"] != "Unknown") & (df_new_3["sub-region"] != "Unknown") & (df_new_3["region"] != "Unknown")]

In [ ]:
# df_new_3 = df_new_3.pivot_table(values=['luc_crops', 'luc_pasture', 'ag_abandon', 'wood_harvest'], index=['country', 'sub-region', 'region'], columns=['time'])

In [ ]:
# df_index_2 = df_new_3.stack(level=0)

In [ ]:
# data = pd.to_datetime(df_index_2.columns, format='%d/%m/%Y %H.%M.%S')
# df_index_2.columns = data.year

In [ ]:
# # df_index.index.name ='type'
# df_index_2.index.set_names(['country', 'sub-region', 'region', 'type'], inplace=True)
# df_index_2 = df_index_2.reset_index()

In [ ]:
# df_index_2.to_excel("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_6CAT_5tn_run16.xlsx", index=False)